In [2]:
import pandas as pd

df = pd.read_csv('BT0_Armory_Transcendent.csv')

In [3]:
def get_candles_day(df):
    df_sales = df[df['eventName'] == 'sale']
    df_sales['date'] = pd.to_datetime(df_sales['date'])
    df_candles = df_sales.groupby(pd.Grouper(key='date', freq='D')).agg({'price': ['first', 'last', 'min', 'max', 'sum']})
    df_candles.columns = ['Open', 'Close', 'Low', 'High', 'Volume']
    return df_candles

df_candles = get_candles_day(df)


/var/folders/cb/lqbl5ztx6fsbpghrl6r70d5m0000gn/T/ipykernel_76680/2164083020.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sales['date'] = pd.to_datetime(df_sales['date'])


In [4]:
csv_files = ['BT0_Time_Warden_Exotic.csv',
            'BT0_Armory_Transcendent.csv',
            'BT0_MysteryBox_Exclusive.csv']

box_dict = {}

for csv_file in csv_files:
    df = pd.read_csv(csv_file)
    box_dict[csv_file[:-4]] = {'volumne': df['price'].sum(), 
                          'burn_count': len(df[df['eventName'] == 'burn']),
                          'sale_count': len(df[df['eventName'] == 'sale']),
                          'supply': len(df['issuedId'].unique())
                          }

box_dict['BT0_Time_Warden_Exotic']['max_supply'] = 500
box_dict['BT0_Armory_Transcendent']['max_supply'] = 450
box_dict['BT0_MysteryBox_Exclusive']['max_supply'] = 2000

for csv_file in csv_files:
    #box_dict[csv_file[:-4]]['inflation'] = box_dict[csv_file[:-4]]['max_supply'] - box_dict[csv_file[:-4]]['supply']
    box_dict[csv_file[:-4]]['circ_supply'] = box_dict[csv_file[:-4]]['supply'] - box_dict[csv_file[:-4]]['burn_count']

df_stats = pd.DataFrame.from_dict(box_dict, orient='index')
df_stats.sort_values(by=['volumne'], inplace=True, ascending=False)
print(df_stats)


                           volumne  burn_count  sale_count  supply  \
BT0_Time_Warden_Exotic    68129.00           0          13      52   
BT0_Armory_Transcendent   29843.99           0           9      37   
BT0_MysteryBox_Exclusive  25379.71        1557        1003    1784   

                          max_supply  circ_supply  
BT0_Time_Warden_Exotic           500           52  
BT0_Armory_Transcendent          450           37  
BT0_MysteryBox_Exclusive        2000          227  
